# Feagent 多 Agent 协作系统演示

本 Notebook 演示 Feagent V2 多 Agent 协作系统的完整工作流程，包括：

1. Agent 初始化与配置
2. 工作流执行与事件处理
3. 执行总结生成
4. 八段压缩与上下文查询

---

## 1. 环境准备

In [ ]:
import sys
from datetime import datetime
from pathlib import Path

# 添加项目根目录到路径
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"项目根目录: {project_root}")
print(f"Python 版本: {sys.version}")

## 2. 导入核心模块

In [ ]:
# 核心 Agent 模块
# 通信模块
from src.domain.agents.agent_channel import (
    AgentChannelBridge,
    AgentMessage,
    AgentMessageType,
    AgentWebSocketChannel,
)
from src.domain.agents.conversation_agent import ConversationAgent
from src.domain.agents.coordinator_agent import CoordinatorAgent

# 执行总结模块
from src.domain.agents.execution_summary import (
    ExecutionError,
    ExecutionSummary,
    NodeExecutionDetail,
)
from src.domain.agents.workflow_agent import WorkflowAgent

# 八段压缩模块
from src.domain.services.power_compressor import (
    PowerCompressor,
)

print("✅ 模块导入成功")

## 3. 创建 Agent 实例

Feagent 采用三层 Agent 架构：
- **CoordinatorAgent**: 系统协调者，管理全局状态
- **ConversationAgent**: 对话管理，处理用户交互
- **WorkflowAgent**: 工作流执行引擎

In [ ]:
# 创建 Agent 实例
coordinator = CoordinatorAgent()
conversation = ConversationAgent()
workflow = WorkflowAgent()

print("✅ Agent 实例创建成功")
print(f"   - CoordinatorAgent: {type(coordinator).__name__}")
print(f"   - ConversationAgent: {type(conversation).__name__}")
print(f"   - WorkflowAgent: {type(workflow).__name__}")

## 4. 模拟工作流执行

创建一个模拟的工作流执行场景，包含多个节点和执行结果。

In [ ]:
# 模拟工作流定义
workflow_definition = {
    "workflow_id": "demo_workflow_001",
    "name": "数据处理演示工作流",
    "nodes": [
        {"id": "node_1", "type": "HTTP", "name": "获取数据"},
        {"id": "node_2", "type": "LLM", "name": "分析数据"},
        {"id": "node_3", "type": "JAVASCRIPT", "name": "转换格式"},
        {"id": "node_4", "type": "HTTP", "name": "保存结果"},
    ],
    "edges": [
        {"source": "node_1", "target": "node_2"},
        {"source": "node_2", "target": "node_3"},
        {"source": "node_3", "target": "node_4"},
    ],
}

print("📋 工作流定义:")
print(f"   ID: {workflow_definition['workflow_id']}")
print(f"   名称: {workflow_definition['name']}")
print(f"   节点数: {len(workflow_definition['nodes'])}")
for node in workflow_definition["nodes"]:
    print(f"      - [{node['type']}] {node['name']}")

In [ ]:
# 创建节点执行详情
node_details = [
    NodeExecutionDetail(
        node_id="node_1",
        node_name="获取数据",
        node_type="HTTP",
        status="completed",
        started_at=datetime.now(),
        completed_at=datetime.now(),
        duration_ms=1250,
        input_data={"url": "https://api.example.com/data"},
        output_data={"records": 100, "size_kb": 45.2},
    ),
    NodeExecutionDetail(
        node_id="node_2",
        node_name="分析数据",
        node_type="LLM",
        status="completed",
        started_at=datetime.now(),
        completed_at=datetime.now(),
        duration_ms=3500,
        input_data={"model": "gpt-4", "prompt_tokens": 500},
        output_data={"analysis": "数据质量良好", "completion_tokens": 200},
    ),
    NodeExecutionDetail(
        node_id="node_3",
        node_name="转换格式",
        node_type="JAVASCRIPT",
        status="failed",
        started_at=datetime.now(),
        completed_at=datetime.now(),
        duration_ms=150,
        input_data={"format": "json"},
        error_message="TypeError: Cannot read property 'map' of undefined",
    ),
    NodeExecutionDetail(
        node_id="node_4",
        node_name="保存结果",
        node_type="HTTP",
        status="skipped",
        started_at=datetime.now(),
        completed_at=datetime.now(),
        duration_ms=0,
        skip_reason="前置节点失败",
    ),
]

print("📊 节点执行状态:")
for detail in node_details:
    status_emoji = {"completed": "✅", "failed": "❌", "skipped": "⏭️"}.get(detail.status, "❓")
    print(f"   {status_emoji} {detail.node_name} ({detail.node_type}) - {detail.duration_ms}ms")

## 5. 生成执行总结

ExecutionSummary 汇总工作流执行的所有信息。

In [ ]:
# 创建执行错误
errors = [
    ExecutionError(
        error_code="JAVASCRIPT_ERROR",
        error_message="TypeError: Cannot read property 'map' of undefined",
        node_id="node_3",
        timestamp=datetime.now(),
        stack_trace="at transform.js:15:10\n    at Array.map (<anonymous>)",
    )
]

# 创建执行总结
summary = ExecutionSummary(
    workflow_id="demo_workflow_001",
    session_id="session_demo_001",
    success=False,
    total_duration_ms=4900,
    started_at=datetime.now(),
    completed_at=datetime.now(),
    node_count=4,
    completed_count=2,
    failed_count=1,
    skipped_count=1,
    node_details=node_details,
    errors=errors,
    metadata={"triggered_by": "user", "priority": "high"},
)

print("📝 执行总结:")
print(f"   工作流ID: {summary.workflow_id}")
print(f"   执行状态: {'成功' if summary.success else '失败'}")
print(f"   总耗时: {summary.total_duration_ms}ms")
print(
    f"   节点统计: {summary.completed_count}完成 / {summary.failed_count}失败 / {summary.skipped_count}跳过"
)
print(f"   错误数: {len(summary.errors)}")

## 6. 八段压缩处理

PowerCompressor 将执行总结压缩为结构化的八段上下文，便于后续分析和对话。

In [ ]:
# 创建压缩器
compressor = PowerCompressor()

print("🗜️ PowerCompressor 八段压缩器")
print("   压缩器已初始化")

In [ ]:
# 执行压缩
compressed = compressor.compress(summary)

print("📦 压缩结果:")
print(f"   类型: {type(compressed).__name__}")
print(f"   工作流ID: {compressed.workflow_id}")
print(f"   会话ID: {compressed.session_id}")
print(f"   压缩时间: {compressed.compressed_at}")

In [ ]:
# 查看八个段落
print("\n📋 八段压缩内容:")
print("\n" + "=" * 60)

print("\n【段落 1: 任务概要】")
print(f"   {compressed.task_summary}")

print("\n【段落 2: 执行状态】")
print(f"   {compressed.execution_status}")

print("\n【段落 3: 完成的子任务】")
for task in compressed.completed_subtasks:
    print(f"   ✅ {task}")

print("\n【段落 4: 子任务错误】")
for error in compressed.subtask_errors:
    print(f"   ❌ [{error.error_type}] {error.node_id}: {error.message}")

print("\n【段落 5: 未解决问题】")
for issue in compressed.unresolved_issues:
    print(f"   ⚠️ {issue.description} (来源: {issue.source_node})")

print("\n【段落 6: 关键发现】")
for finding in compressed.key_findings:
    print(f"   💡 {finding}")

print("\n【段落 7: 建议的下一步】")
for step in compressed.next_steps:
    print(f"   ➡️ [{step.priority}] {step.description}")

print("\n【段落 8: 知识引用】")
for ref in compressed.knowledge_references:
    print(f"   📚 {ref.title} ({ref.source_type})")

print("\n" + "=" * 60)

## 7. 使用 CoordinatorAgent 管理压缩上下文

In [ ]:
# 将压缩结果存储到 Coordinator
coordinator.store_compressed_context(workflow_id=summary.workflow_id, data=compressed.to_dict())

print("💾 压缩上下文已存储到 CoordinatorAgent")

In [ ]:
# 查询子任务错误
errors = coordinator.query_subtask_errors("demo_workflow_001")

print("🔍 查询子任务错误:")
for error in errors:
    print(f"   - [{error['error_type']}] {error['node_id']}: {error['message']}")

In [ ]:
# 查询未解决问题
issues = coordinator.query_unresolved_issues("demo_workflow_001")

print("🔍 查询未解决问题:")
for issue in issues:
    print(f"   - {issue['description']} (严重程度: {issue['severity']})")

In [ ]:
# 查询建议的下一步
next_plan = coordinator.query_next_plan("demo_workflow_001")

print("🔍 查询建议的下一步:")
for step in next_plan:
    print(f"   - [{step['priority']}] {step['description']}")

In [ ]:
# 获取对话上下文
context = coordinator.get_context_for_conversation("demo_workflow_001")

print("💬 对话上下文:")
print(f"   任务概要: {context['task_summary']}")
print(f"   执行状态: {context['execution_status']}")
print(f"   完成的子任务数: {len(context['completed_subtasks'])}")
print(f"   错误数: {len(context['subtask_errors'])}")

In [ ]:
# 查看压缩统计
stats = coordinator.get_power_compression_statistics()

print("📊 压缩统计:")
print(f"   总压缩次数: {stats['total_compressions']}")
print(f"   存储的上下文数: {stats['stored_contexts']}")

## 8. WebSocket 通信演示

演示 Agent 间的 WebSocket 通信机制。

In [ ]:
# 创建通信信道
channel = AgentWebSocketChannel()
bridge = AgentChannelBridge(channel=channel)

print("🔌 WebSocket 通信组件:")
print("   - AgentWebSocketChannel: 管理 WebSocket 连接")
print("   - AgentChannelBridge: Agent 间通信桥接")

In [ ]:
# 演示消息创建
progress_message = AgentMessage(
    type=AgentMessageType.EXECUTION_PROGRESS,
    session_id="session_demo_001",
    payload={
        "workflow_id": "demo_workflow_001",
        "current_node": "node_2",
        "progress": 0.5,
        "message": "正在分析数据...",
    },
)

print("📨 消息示例:")
print(f"   类型: {progress_message.type.value}")
print(f"   会话: {progress_message.session_id}")
print(f"   消息ID: {progress_message.message_id}")
print(f"   负载: {progress_message.payload}")

In [ ]:
import json

# 消息序列化
message_dict = progress_message.to_dict()

print("📤 序列化后的消息:")
print(json.dumps(message_dict, indent=2, default=str, ensure_ascii=False))

## 9. 完整工作流程总结

以下是 Feagent 多 Agent 系统的完整工作流程：

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║                 Feagent 多 Agent 协作系统流程                      ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  1. 用户请求 ──────────────────────────────────────────────────► ║
║     │                                                            ║
║     ▼                                                            ║
║  2. ConversationAgent 接收请求，解析意图                           ║
║     │                                                            ║
║     ▼                                                            ║
║  3. CoordinatorAgent 协调工作流分发                               ║
║     │                                                            ║
║     ▼                                                            ║
║  4. WorkflowAgent 执行工作流节点                                  ║
║     │   ├─ HTTP 节点：API 调用                                    ║
║     │   ├─ LLM 节点：模型推理                                     ║
║     │   ├─ JavaScript 节点：数据转换                              ║
║     │   └─ 更多节点类型...                                        ║
║     │                                                            ║
║     ▼                                                            ║
║  5. 生成 ExecutionSummary 执行总结                                ║
║     │                                                            ║
║     ▼                                                            ║
║  6. PowerCompressor 八段压缩                                      ║
║     │   ├─ 任务概要                                              ║
║     │   ├─ 执行状态                                              ║
║     │   ├─ 完成的子任务                                          ║
║     │   ├─ 子任务错误                                            ║
║     │   ├─ 未解决问题                                            ║
║     │   ├─ 关键发现                                              ║
║     │   ├─ 建议的下一步                                          ║
║     │   └─ 知识引用                                              ║
║     │                                                            ║
║     ▼                                                            ║
║  7. CoordinatorAgent 存储压缩上下文                               ║
║     │                                                            ║
║     ▼                                                            ║
║  8. ConversationAgent 查询上下文，生成响应                         ║
║     │                                                            ║
║     ▼                                                            ║
║  9. 用户收到响应 ◄──────────────────────────────────────────────  ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
""")

## 10. 下一步探索

- **API 文档**: 访问 `http://localhost:8000/docs` 查看完整 API
- **架构文档**: 阅读 `docs/architecture/multi_agent_collaboration_guide.md`
- **运维指南**: 参考 `docs/operations/operations_guide.md`
- **测试套件**: 运行 `pytest tests/` 了解更多用例

In [ ]:
print("\n🎉 演示完成！")
print("\n感谢使用 Feagent 多 Agent 协作系统。")